# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298425516844                   -0.85    4.0
  2   -8.300208383005       -2.75       -1.25    1.0
  3   -8.300449447270       -3.62       -1.89    3.2
  4   -8.300462814657       -4.87       -2.77    3.0
  5   -8.300464380138       -5.81       -3.09    6.4
  6   -8.300464497788       -6.93       -3.23    5.9
  7   -8.300464566166       -7.17       -3.36    1.9
  8   -8.300464604568       -7.42       -3.50    1.0
  9   -8.300464632508       -7.55       -3.68    1.0
 10   -8.300464636063       -8.45       -3.75    2.6
 11   -8.300464642058       -8.22       -3.97    1.2
 12   -8.300464643978       -8.72       -4.51    1.6


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64211146642                   -0.70    6.1
  2   -16.67867160488       -1.44       -1.14    2.1
  3   -16.67923216587       -3.25       -1.86    2.5
  4   -16.67927676286       -4.35       -2.74    2.2
  5   -16.67928565724       -5.05       -3.16    5.8
  6   -16.67928621588       -6.25       -3.54    2.5
  7   -16.67928622234       -8.19       -4.02    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32453937266                   -0.56    6.5
  2   -33.33306098460       -2.07       -0.99    1.0
  3   -33.33597988869       -2.53       -1.70    4.1
  4   -33.33686476806       -3.05       -2.36    4.5
  5   -33.33693830418       -4.13       -2.94    8.1
  6   -33.33694388756       -5.25       -3.89    3.9
  7   -33.33694392367       -7.44       -4.29   11.5


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298629197463                   -0.85    4.6
  2   -8.300290741902       -2.78       -1.59    1.0
  3   -8.300451041823       -3.80       -2.55    6.0
  4   -8.300382595242   +   -4.16       -2.30   13.6
  5   -8.300464241177       -4.09       -3.49    3.1
  6   -8.300464576985       -6.47       -3.89    4.5
  7   -8.300464642278       -7.19       -4.69    3.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32455450154                   -0.56    6.1
  2   -33.31728402694   +   -2.14       -1.27    1.0
  3   -16.81376504853   +    1.22       -0.46    7.6
  4   -33.32677959794        1.22       -1.95    7.8
  5   -33.27091110603   +   -1.25       -1.55    3.5
  6   -33.08582780010   +   -0.73       -1.36    3.9
  7   -33.31384670709       -0.64       -1.80    4.0
  8   -33.33652891143       -1.64       -2.51    3.2
  9   -33.33674789888       -3.66       -2.67    3.9
 10   -33.33675785147       -5.00       -2.62    2.6
 11   -33.33692374305       -3.78       -3.30    2.4
 12   -33.33694165468       -4.75       -3.61    3.2
 13   -33.33694355302       -5.72       -3.90    3.2
 14   -33.33694385909       -6.51       -4.21    4.2


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.